# Set Partitioning Problem
A set partitioning problem determines how the items in one set (S) can be partitioned into smaller subsets. All items in S must be contained in one and only one partition. Related problems are:

- set packing - all items must be contained in zero or one partitions;
- set covering - all items must be contained in at least one partition.

In this case study a wedding planner must determine guest seating allocations for a wedding. To model this problem the tables are modelled as the partitions and the guests invited to the wedding are modelled as the elements of S. The wedding planner wishes to maximise the total happiness of all of the tables. 

A set partitioning problem may be modelled by explicitly enumerating each possible subset. Though this approach does become intractable for large numbers of items (without using column generation) it does have the advantage that the objective function co-efficients for the partitions can be non-linear expressions (like happiness) and still allow this problem to be solved using Linear Programming.

From: https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html#set-partitioning-problem

In [4]:
import pulp as pl

In [1]:
max_tables = 2
max_table_size = 8
# guests = "A B C D E F G H I J K L M N O P Q R S".split()
guests = "A B C D".split()

In [2]:
def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))

In [5]:
# create list of all possible tables
possible_tables = [tuple(c) for c in pl.allcombinations(guests, max_table_size)]

In [6]:
# create a binary variable to state that a table setting is used
x = pl.LpVariable.dicts(
    "table", possible_tables, lowBound=0, upBound=1, cat=pl.LpInteger
)

We create the LpProblem and then make the objective function. Note that happiness function used in this script would be difficult to model in any other way.

In [61]:
seating_model = pl.LpProblem("Wedding Seating Model", pl.LpMinimize)

seating_model += pl.lpSum([happiness(table) * x[table] for table in possible_tables])

Her samles det i modellen:
- x er array af alle pladser

In [7]:
print(x)

{('A',): table_('A',), ('B',): table_('B',), ('C',): table_('C',), ('D',): table_('D',), ('A', 'B'): table_('A',_'B'), ('A', 'C'): table_('A',_'C'), ('A', 'D'): table_('A',_'D'), ('B', 'C'): table_('B',_'C'), ('B', 'D'): table_('B',_'D'), ('C', 'D'): table_('C',_'D'), ('A', 'B', 'C'): table_('A',_'B',_'C'), ('A', 'B', 'D'): table_('A',_'B',_'D'), ('A', 'C', 'D'): table_('A',_'C',_'D'), ('B', 'C', 'D'): table_('B',_'C',_'D'), ('A', 'B', 'C', 'D'): table_('A',_'B',_'C',_'D')}


Possible_tables indeholder et array af alle kombinationerne, fx bord 0 er ('A'), bord 100 er ('F', 'H') og bord 10000 er ('B', 'H', 'I', 'L', 'P')

In [74]:
print(possible_tables[0])

('A',)


In [62]:
# specify the maximum number of tables
seating_model += (
    pl.lpSum([x[table] for table in possible_tables]) <= max_tables,
    "Maximum_number_of_tables",
)

In [63]:
# A guest must seated at one and only one table
for guest in guests:
    seating_model += (
        pl.lpSum([x[table] for table in possible_tables if guest in table]) == 1,
        f"Must_seat_{guest}",
    )

In [64]:
# I've taken the optimal solution from a previous solving. x is the variable dictionary.
# solution = {
#     ("M", "N"): 1.0,
#     ("E", "F", "G"): 1.0,
#     ("A", "B", "C", "D"): 1.0,
#     ("I", "J", "K", "L"): 1.0,
#     ("O", "P", "Q", "R"): 1.0,
# }
# for k, v in solution.items():
#     x[k].setInitialValue(v)

In [65]:
solver = pl.PULP_CBC_CMD(msg=True, warmStart=True)
# solver = pulp.CPLEX_CMD(msg=True, warmStart=True)
# solver = pulp.GUROBI_CMD(msg=True, warmStart=True)
# solver = pulp.CPLEX_PY(msg=True, warmStart=True)
# solver = pulp.GUROBI(msg=True, warmStart=True)
seating_model.solve(solver)

1

In [66]:
print(f"The choosen tables are out of a total of {len(possible_tables)}:")
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)

The choosen tables are out of a total of 169765:
('O', 'P', 'Q', 'R', 'S')
('A', 'B', 'C', 'D', 'E', 'F', 'G')
('H', 'I', 'J', 'K', 'L', 'M', 'N')
